In [1]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
import nltk
import string
import numpy as np
import re


In [2]:
# Dataset Retrieval And Processing

import pandas as pd
# TF-IDF(document) = TF-IDF(title) * alpha + TF-IDF(body) * (1-alpha)
Name = "Dataset"
alpha = 0.3 # Weight

dat_csv = pd.read_csv("/Users/srikaramara/Desktop/pdb_project/dat.csv")
dat_csv = dat_csv.iloc[3:]
dat_csv = dat_csv.reset_index()
dat_csv = dat_csv.drop(columns = ['index'])

In [ ]:
dat_csv['Filename'] = '/Users/srikaramara/Desktop/pdb_project/Dataset/' + dat_csv['Filename'].astype(str)
dataset = dat_csv.to_records(index=False)
dataset = list(dataset)

In [13]:
dataset

[('/Users/srikaramara/Desktop/pdb_project/Dataset/100west.txt', 'Going 100 West by 53 North by Jim Prentice (1990)'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/13chil.txt', 'The Story of the Sly Fox'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/14.lws', 'A Smart Bomb with a Language Parser'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/16.lws', 'Two Guys in a Garage, by M. Pshota'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/17.lws', 'The Early Days of a High-Tech Start-up are Magic (November 18, 1991) by M. Peshota'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/18.lws', 'The Couch, the File Cabinet, and the Calendar, by M. Peshota (December 9, 1991)'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/19.lws', 'Engineering the Future of American Technology by M. Peshota (January 5, 1992)'),
 ('/Users/srikaramara/Desktop/pdb_project/Dataset/20.lws', 'What Research and Development Was Always Meant to Be, by M. Peshota'),
 ('/Users/srikaramara/Desktop/pd

In [3]:
def Search_document(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r')
    Data = file.read()
    file.close()
    print("\n------Retrieved data--------\n", Data)

In [4]:
Search_document(40)

('/Users/srikaramara/Desktop/pdb_project/Dataset/arcadia.sty', 'Arcadia, a Story')

------Retrieved data--------
 
Story written by: lim@merck.com

     In the early morning light, the wide Alderaani forests were
tipped with gold. Danah Antilles, Dowager Princess of Alderaan,
surveyed the dappled pelt of gilt leaves and shadow from high
above. She sat at her breakfast table as if enthroned. Although
she was nearly ninety, her back was sapling-straight, and her
eyes were as clear as the jewels clasped around her throat.
Another strand of gems shone above them, wound into her silver
hair. On the other side of the prismed tower chamber, her son
nodded to the servant droids, dismissing them.
     Danah turned away from the view, waiting for the soft drone
of the elevator shaft to diminish and fall silent. "Well," she
said at last, "What is the news from the Senate?"
     "Bothersome," Bail Organa said. His pale blue robes flowed
behind him as he walked back to his own chair. Sitting down, 

In [9]:
# Clean Text
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)


from nltk.tokenize import word_tokenize
pr_text = []
pr_title = []

for i in dataset[:len(dataset)]:
    file = open(i[0], 'r',errors='ignore')
    text = file.read()
    file.close()

    pr_text.append(word_tokenize(str(clean_text(text))))
    pr_title.append(word_tokenize(str(clean_text(i[1]))))

In [18]:
type(pr_text[0][0])

str

In [19]:
pr_title

[['go', 'west', 'north', 'jim', 'prentice'],
 ['story', 'sly', 'fox'],
 ['smart', 'bomb', 'language', 'parser'],
 ['two', 'guy', 'garage', 'pshota'],
 ['early', 'day', 'high-tech', 'start-up', 'magic', 'november', 'peshota'],
 ['couch', 'file', 'cabinet', 'calendar', 'peshota', 'december'],
 ['engineering', 'future', 'american', 'technology', 'peshota', 'january'],
 ['research', 'development', 'always', 'mean', 'peshota'],
 ['adventure', 'three', 'gable'],
 ['story', 'little', 'pig'],
 ['collection', 'sonnet', 'staeorra', 'rokraven', 'march'],
 ['adventure', 'three', 'student'],
 ['story', 'three', 'wish'],
 ['fiction', 'moon', 'death'],
 ['story', 'five', 'orange', 'pip'],
 ['story', 'six', 'able', 'men'],
 ['adventure', 'six', 'napoleon'],
 ['seven', 'old', 'samurai'],
 ['seven', 'voyage', 'sinbad', 'sailor'],
 ['story', 'ali', 'baba', 'thief'],
 ['adventure', 'abbey', 'grange', 'rp', 'story'],
 ['life', 'go', 'edge', 'abyss'],
 ['chronicle', 'astrus', 'ii', 'father', 'adler', 'ben',

## Calculating Data Frequency for all words

In [30]:
Data_Frequency = {}

for i in range(len(dataset)):
    tokens = pr_text[i]
    for w in tokens:
        try:
            Data_Frequency[w].add(i)
        except:
            Data_Frequency[w] = {i}

    tokens = pr_title[i]
    for w in tokens:
        try:
            Data_Frequency[w].add(i)
        except:
            Data_Frequency[w] = {i}
for i in Data_Frequency:
    Data_Frequency[i] = len(Data_Frequency[i])

In [31]:
Data_Frequency

{'shareware': 5,
 'trial': 32,
 'project': 52,
 'freeware': 1,
 'need': 229,
 'support': 69,
 'continue': 154,
 'west': 62,
 'north': 71,
 'by': 209,
 'jim': 20,
 'prentice': 3,
 'copyright': 110,
 'brandon': 3,
 'manitoba': 2,
 'canada': 10,
 'magic': 73,
 'phrase': 36,
 'speak': 158,
 'mumble': 40,
 'thought': 231,
 'inwardly': 8,
 'thousand': 97,
 'souls': 17,
 'venture': 52,
 'northward': 7,
 'an': 178,
 'imaginary': 20,
 'line': 169,
 'show': 213,
 'map': 45,
 'label': 19,
 'degrees': 11,
 '.': 442,
 'presence': 81,
 'indicate': 45,
 'highway': 34,
 'traveller': 18,
 'road': 115,
 'side': 226,
 'signs': 16,
 'division': 21,
 'territory': 17,
 'distinct': 27,
 'mind': 217,
 'any': 95,
 'international': 18,
 'border': 31,
 'been': 157,
 '!': 313,
 'writer': 52,
 'poet': 16,
 'pilot': 26,
 'have': 194,
 'contribute': 21,
 'lore': 6,
 'rigor': 2,
 'life': 276,
 'in': 320,
 'bush': 44,
 'tell': 296,
 'tale': 66,
 'man': 259,
 'eat': 130,
 'mosquito': 6,
 'of': 330,
 'murderous': 13,
 '

In [32]:
total_vocab_size = len(Data_Frequency)

In [33]:
total_vocab_size

53224

In [34]:
total_vocab = [x for x in Data_Frequency]

In [35]:
print(total_vocab[:20])

['shareware', 'trial', 'project', 'freeware', 'need', 'support', 'continue', 'west', 'north', 'by', 'jim', 'prentice', 'copyright', 'brandon', 'manitoba', 'canada', 'magic', 'phrase', 'speak', 'mumble']


In [36]:
def doc_freq(word):
    c = 0
    try:
        c = Data_Frequency[word]
    except:
        pass
    return c

## Calculating Tf-Idf

In [74]:
document = 0

term_freq_invdocfreq = {}

for i in range(len(dataset)):
    
    tokens = pr_text[i]
    
    counter = Counter(tokens + pr_title[i])
    words_count = len(tokens + pr_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((len(dataset)+1)/(df+1))
        
        term_freq_invdocfreq[document, token] = tf*idf

    document += 1

In [84]:
document = 0

term_freq_invdocfreq_title = {}

for i in range(len(dataset)):
    
    tokens = pr_title[i]
    counter = Counter(tokens + pr_text[i])
    words_count = len(tokens + pr_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((len(dataset)+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        term_freq_invdocfreq_title[document, token] = tf*idf

    document += 1

In [85]:
term_freq_invdocfreq[(0,"go")]

0.00015225763737604676

In [86]:
term_freq_invdocfreq_title[(0,"go")]

0.00015225763737604676

## Merging Tf-idf 

In [87]:
for i in term_freq_invdocfreq:
    term_freq_invdocfreq[i] *= alpha

In [88]:
for i in term_freq_invdocfreq_title:
    term_freq_invdocfreq[i] = term_freq_invdocfreq_title[i]

In [89]:
len(term_freq_invdocfreq)

390610

In [90]:
term_freq_invdocfreq

{(0, '!'): 5.5897174312371215e-05,
 (0, "''"): 2.882045239135076e-05,
 (0, "'s"): 5.225211980433869e-05,
 (0, ')'): 0.00019395603035451592,
 (0, ','): 0.0007009296208425368,
 (0, '.'): 0.00033364374675862505,
 (0, '..'): 0.00038204995620317374,
 (0, ':'): 0.0002571100865030871,
 (0, '?'): 0.00016623667411879046,
 (0, 'a'): 0.0001435846298417897,
 (0, 'able'): 0.00014419036051523367,
 (0, 'abound'): 0.0005302747528013892,
 (0, 'absence'): 0.00038204995620317374,
 (0, 'access'): 0.0003627182268225215,
 (0, 'accustom'): 0.000936869248935316,
 (0, 'activity'): 0.0007182588743638448,
 (0, 'acute'): 0.0005815923282901354,
 (0, 'adapt'): 0.00047587593064499895,
 (0, 'add'): 0.00022152308561834778,
 (0, 'adjust'): 0.0003949463549873217,
 (0, 'adults'): 0.0006359911504465258,
 (0, 'advance'): 0.00030831940466613116,
 (0, 'adventuresome'): 0.0008270582020726622,
 (0, 'afford'): 0.00033014464772266493,
 (0, 'ago'): 0.00016064520983061524,
 (0, 'aid'): 0.00032438862413605897,
 (0, 'air'): 0.000258

# TF-IDF Max Scoring

In [104]:
def Max_score(k, termsArray):
    preprocessed_query = clean_text(termsArray)
    
    tokens = preprocessed_query.split()

    print("Max_score")
    print("\n Given Query :", termsArray)
    print("")
    
    query_weights = {}

    for key in term_freq_invdocfreq:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += term_freq_invdocfreq[key]
            except:
                query_weights[key[0]] = term_freq_invdocfreq[key]
                
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

Max_score(10, "Falling into the system where life becomes ever so painful,The frustration of not knowing who is whom, what is that, why is this Dateline: Age 4 - Religious leaders spout sayings of eternal damnation I spin through cycles of pitfalls and laugh at their blind nature")

Max_score

 Given Query : Falling into the system where life becomes ever so painful,The frustration of not knowing who is whom, what is that, why is this Dateline: Age 4 - Religious leaders spout sayings of eternal damnation I spin through cycles of pitfalls and laugh at their blind nature

[323, 361, 10, 345, 417, 117, 53, 17, 28, 67]


In [105]:
Search_document(323)

('/Users/srikaramara/Desktop/pdb_project/Dataset/plescopm.txt', 'Asleep by Craig Plesco')

------Retrieved data--------
 

ASLEEP - Craig Plesco

I'm asleep and my eyes see reality....

I'm asleep and my mind focuses on the life I could never indulge in...

Petty concerns split my neuro-system/Death becomes but a joke

Falling into the system where life becomes ever so painful,
The frustration of not knowing who is whom, what is that, why is this
Dateline: Age 4 - Religious leaders spout sayings of eternal damnation
I spin through cycles of pitfalls and laugh at their blind nature

What determines life?
How soon after 'death' does one become useless in life?
When the moistness dries, when the water subsides...
Is water life? For beings such as we, perhaps this is true....

Water/Sun.....Cold/Hot.....
What is where, I am not..

I feel parched, feeling too much heat
Sporadically I am fed tiny droplets of life...
I drink and savor and enjoy...even basking in the heat
Then a yearning creep

## Top-k OR Function

In [67]:
def top_k(k, termsArray):
    preprocessed_query = clean_text(termsArray)
    
    tokens = preprocessed_query.split()
    tokens.sort()
    print(tokens)
    
top_k (2,"hello srikar amara")

['amara', 'hello', 'srikar']
